In [1]:
import sys
sys.path.append('../../')
from utils.preprocess import PreProcess
from sklearn.model_selection import train_test_split
import html
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm
import numpy as np
from sklearn.feature_selection import SelectKBest

ModuleNotFoundError: No module named 'utils'

In [ ]:
from src.data_util import biocaser2text
data_file = "../../data/biocaster/BioCaster.3.xml"
data_df = biocaser2text(data_file)

In [ ]:
data_df.head()

In [ ]:
data_df["Category"].value_counts()

### Preprocessing

In [ ]:
column_name = data_df.columns[0]
    # print(column_name)
pre_processor = PreProcess(data_df, column_name)
# todo: change code to provide all functions in class definition.
data = pre_processor.clean_html()
data = pre_processor.remove_non_ascii()
data = pre_processor.remove_spaces()
data = pre_processor.remove_punctuation()
data = pre_processor.stemming()
data = pre_processor.lemmatization()
data = pre_processor.stop_words()

### Feature extraction

#### TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1)
data["tfidf"] = list(vectorizer.fit_transform(data.Document.to_numpy()).toarray())
len(vectorizer.get_feature_names())

### Bag of Word

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
data["bow"] = list(vectorizer.fit_transform(data.Document.to_numpy()).toarray())
len(vectorizer.get_feature_names())

### Bigram

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(2,2),min_df=2)
data["bigram"] =list(vectorizer.fit_transform(data.Document.to_numpy()).toarray())
len(vectorizer.get_feature_names())

### TriGram

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(3,3),min_df=2)
data["trigram"] = list(vectorizer.fit_transform(data.Document.to_numpy()).toarray())
len(vectorizer.get_feature_names())

In [ ]:
from sklearn.feature_selection import chi2


### TFIDF + Bag of Words

In [ ]:
from scipy.sparse import hstack
X = hstack([data["tfidf"],data["bow"]])

In [215]:
features = ["bow","tfidf","bigram",["bigram","trigram"],["tfidf","bigram"],["bow","bigram"],["tfidf","trigram"],["bow","trigram"]]
for col in features:
    acc = score_NB(data,col)
    print(col,acc)

  0%|          | 0/10 [00:00<?, ?it/s]

bow 0.8955009900990099


  0%|          | 0/10 [00:00<?, ?it/s]

tfidf 0.9002019801980199


  0%|          | 0/10 [00:00<?, ?it/s]

...

['bow', 'bigram'] 0.8938148514851486


  0%|          | 0/10 [00:00<?, ?it/s]

['tfidf', 'trigram'] 0.9018009900990099


100%|██████████| 10/10 [01:05<00:00,  6.59s/it]

['bow', 'trigram'] 0.8930029702970298


In [216]:
for col in features:
    acc = score_svm(data,col)
    print(col,acc)

  0%|          | 0/10 [00:00<?, ?it/s]

bow 0.9011207920792079


  0%|          | 0/10 [00:00<?, ?it/s]

tfidf 0.9234297029702973


  0%|          | 0/10 [00:00<?, ?it/s]

...

['bow', 'bigram'] 0.9016861386138615


  0%|          | 0/10 [00:00<?, ?it/s]

['tfidf', 'trigram'] 0.9130683168316832


100%|██████████| 10/10 [07:11<00:00, 43.19s/it]

['bow', 'trigram'] 0.9065821782178218


In [217]:
for col in features:
    acc = score_id3(data,col)
    print(col,acc)

  0%|          | 0/10 [00:00<?, ?it/s]

bow 0.8342168316831682


  0%|          | 0/10 [00:00<?, ?it/s]

tfidf 0.8210178217821781


  0%|          | 0/10 [00:00<?, ?it/s]

...

['bow', 'bigram'] 0.8245188118811881


  0%|          | 0/10 [00:00<?, ?it/s]

['tfidf', 'trigram'] 0.817660396039604


100%|██████████| 10/10 [01:11<00:00,  7.15s/it]

['bow', 'trigram'] 0.8237217821782178


### SVM

In [211]:
def score_svm(data,col):
    from sklearn.model_selection import cross_val_score
    from sklearn import svm
    from sklearn.utils import shuffle
    clf = svm.SVC(kernel='linear', C=1)

    scores_sum = []
    for i in tqdm(range(10)):
        data=shuffle(data)
        if type(col)==list:
            X = []
            x1 = data[col[0]].to_numpy()
            x2 = data[col[1]].to_numpy()
            for f1,f2 in zip(x1,x2):
                X.append(f1+f1)
        else:
            X = list(data[col])
        ch2 = SelectKBest(chi2, k=9000)
        step = [('sel', ch2), ('cls', clf)]
        clf = Pipeline(step)
        y = data.Category.to_numpy()
        scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
        score_mean = scores.mean()
        scores_sum.append(score_mean)
    return sum(scores_sum) / len(scores_sum)

In [212]:
acc = score_svm(data,"tfidf")
print(col,acc)

100%|██████████| 10/10 [09:18<00:00, 55.80s/it]

['bow', 'trigram'] 0.9250297029702971


## Naive Bayes

In [213]:
def score_NB(data,col):
    from sklearn.model_selection import cross_val_score
    from sklearn import svm
    from sklearn.utils import shuffle
    from sklearn.naive_bayes import BernoulliNB
    clf = BernoulliNB()

    scores_sum = []
    for i in tqdm(range(10)):
        data=shuffle(data)
        if type(col)==list:
            X = []
            x1 = data[col[0]].to_numpy()
            x2 = data[col[1]].to_numpy()
            for f1,f2 in zip(x1,x2):
                X.append(f1+f1)
        else:
            X = list(data[col])
        ch2 = SelectKBest(chi2, k=9000)
        step = [('sel', ch2), ('cls', clf)]
        clf = Pipeline(step)
        y = data.Category
        scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
        score_mean = scores.mean()
        scores_sum.append(score_mean)
    return sum(scores_sum) / len(scores_sum)

### DecisionTreeClassifier ID3

In [214]:
def score_id3(data,col):
    from sklearn.model_selection import cross_val_score
    from sklearn import svm
    from sklearn.utils import shuffle
    from sklearn.tree import DecisionTreeClassifier
    clf = DecisionTreeClassifier(random_state=0,criterion='entropy')

    scores_sum = []
    for i in tqdm(range(10)):
        data=shuffle(data)
        if type(col)==list:
            X = []
            x1 = data[col[0]].to_numpy()
            x2 = data[col[1]].to_numpy()
            for f1,f2 in zip(x1,x2):
                X.append(f1+f1)
        else:
            X = list(data[col])
        ch2 = SelectKBest(chi2, k=9000)
        step = [('sel', ch2), ('cls', clf)]
        clf = Pipeline(step)
        y = data.Category
        scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
        score_mean = scores.mean()
        scores_sum.append(score_mean)
    return sum(scores_sum) / len(scores_sum)

In [ ]:
score_id3(data,'trigram')

### DecisionTreeClassifier C4.5

In [ ]:
def score_c45(data,col):
    from sklearn.model_selection import cross_val_score
    from sklearn import svm
    from sklearn.utils import shuffle
    from c45 import C45
    model = C45()

    scores_sum = []
    for i in tqdm(range(10)):
        data=shuffle(data)
        X = list(data[col])
        y = data.Category
        scores = cross_val_score(model, X, y, cv=10, scoring='accuracy')
        score_mean = scores.mean()
        scores_sum.append(score_mean)
    return sum(scores_sum) / len(scores_sum)

In [ ]:
score_c45(data,'trigram')

In [ ]:
def read_process_data(data_df):
    column_name = data_df.columns[0]
    # print(column_name)
    pre_processor = PreProcess(data_df, column_name)
    # todo: change code to provide all functions in class definition.
    data = pre_processor.clean_html()
    data = pre_processor.remove_non_ascii()
    data = pre_processor.remove_spaces()
    data = pre_processor.remove_punctuation()
    data = pre_processor.stemming()
    data = pre_processor.lemmatization()
    data = pre_processor.stop_words()
    train_x, test_x, train_y, test_y = train_test_split(data.Document, data.Category, test_size=0.20)
    tfidf_transformer = TfidfVectorizer(min_df=1)
    train_vectors = tfidf_transformer.fit_transform(train_x)
    return train_vectors, train_y

In [ ]:
x,y = read_process_data(data_df)

In [ ]:
from sklearn.model_selection import cross_val_score


In [ ]:
# x.todense(),y

In [ ]:
def remove_non_ascii(df):
    df = df.apply(lambda x: "".join(i for i in x if ord(i) < 128))
    return df
class PreProcess(BaseEstimator, TransformerMixin):
    """
    This class contains all text pre-processing function
    # Input parameters: Dataframe, Column_name on which function needs to be applied
    # Output parameters: Return dataframe after applying operations
    """
    # todo: Pass functions as a list of arguments to apply in the class
    # todo: make set of words before applying all operations to reduce processing time.
    def __init__(self):
        self.stemmer = PorterStemmer()
        self.lemmatiser = WordNetLemmatizer()
        # pass

    def clean_html(df):
        """remove html entities"""
        df = df.apply(html.unescape)
        return df

    def remove_spaces(df):
        df = df.apply(lambda x: x.replace('\n', ' '))
        df = df.apply(lambda x: x.replace('\t', ' '))
        df = df.apply(lambda x: x.replace('  ', ' '))
        df = df.apply(lambda x: x.lower())
        return df

    def remove_punctuation(self):
        tr = str.maketrans("", "", string.punctuation)
        # self.data[self.column_name] = self.data[self.column_name].apply(lambda x: " ".join([item.translate(tr)
        #                                                                 for item in x.split()]))
        self.data[self.column_name] = self.data[self.column_name].apply(lambda x: x.translate(tr))
        return self.data

    def stemming(self):
        # todo: provide option of selecting stemmer.
        snowball_stemmer = SnowballStemmer('english')
        # self.data[self.column_name] = self.data[self.column_name].apply(lambda x: " ".join([snowball_stemmer.stem(item)
        #                                                                 for item in x.split()]))
        self.data[self.column_name] = self.data[self.column_name].apply(lambda x: " ".join([self.stemmer.stem(item)
                                                                        for item in x.split()]))
        return self.data

    def lemmatization(self):
        self.data[self.column_name] = self.data[self.column_name].apply(lambda x: " ".join([self.lemmatiser.lemmatize(item)
                                                                        for item in x.split()]))
        return self.data

    def stop_words(self):
        stop = stopwords.words('english')
        self.data[self.column_name] = self.data[self.column_name].apply(lambda x: " ".join(set([item for item in x.split() if
                                                                                       item not in stop])))
        return self.data
    
    def transform(self, df):
        df = remove_non_ascii(df)
        df = clean_html(df)
        return df
        
    def fit(self, df, y=None, **fit_params):
        return self

In [ ]:
df.columns[0]

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class PreProcessing(BaseEstimator, TransformerMixin):
    """Custom Pre-Processing estimator for our use-case
    """

    def __init__(self):
        pass

    def transform(self, df):
        """
        Preprocessing steps for text processing
        """
#         columns = df.columns()
        df = df.apply(lambda x: x.lower())
        df = df.apply(lambda x: "".join(i for i in x if ord(i) < 128))
        df = df.apply(html.unescape)
        return df

    def fit(self, df, y=None, **fit_params):
        return self

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC

In [ ]:
pipe = Pipeline([
    ('Test1', PreProcess())
])

In [ ]:
xx =pipe.fit_transform(df.Document)

In [ ]:
xx

In [ ]:
df.Document

In [ ]:
import pandas as pd

data = [{'A': [1,2,3,4], 'B': [2,3,4,5], 'C': [4,5,5,6], 'D': [6,3,4,5]}, {'A': [2,3,5,6], 'B': [3,4,6,6], 'C': [3,4,5,7], 'D': [2,6,3,4]}, {'A': [8,9,6,7], 'B': [5,7,9,5], 'C': [3,7,9,5], 'D': [5,7,9,8]}]
df = pd.DataFrame(data)
df

In [ ]:
df["E"]=np.array([[1,2,3,4],  [2,3,4,5], [4,5,5,6]])
df

In [2]:
!hostname

mmll0537
